
# Data Processing and Extraction using NLP
This notebook processes textual data, identifies important sentences using Natural Language Processing (NLP), 
and exports the results to a CSV file. The dataset used contains various text snippets, and our goal is to 
extract important information by recognizing entities and other key aspects.


In [14]:
import pandas as pd
import re
import spacy
from langdetect import detect, DetectorFactory

# Load spaCy model (Dutch language model for this case)
nlp = spacy.load("nl_core_news_lg")

# Enable GPU usage for spaCy if available
spacy.prefer_gpu()


False

In [15]:
# Load the dataset
try:
    df = pd.read_csv('Antwerpen_c_2.csv')
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: Dataset file not found.")

Dataset loaded successfully.


In [16]:
# Split the text into sentences using regex for sentence boundary detection
def split_sentences(text):
    # Pattern matches sentence boundaries after punctuation followed by a capital letter
    sentence_pattern = r'(?<=[.!?])\s+(?=[A-Z])'
    sentences = re.split(sentence_pattern, text)
    return sentences

# Define a function to determine if a sentence is important
def is_important(sentence):
    doc = nlp(sentence)
    
    # Check for named entities
    if len(doc.ents) > 0:
        return True
    
    # Check if the sentence length is within a reasonable range
    if 3 < len(doc) < 30:
        return True

    # Check for specific parts of speech such as proper nouns or numbers
    if any(token.pos_ == 'PROPN' or token.pos_ == 'NUM' for token in doc):
        return True
    
    return False



# Ensure consistent results
DetectorFactory.seed = 0

def is_dutch(text):
    try:
        language = detect(text)
        if language == 'nl':
            return True
        else:
            return False
    except:
        return False

In [ ]:

# Process all sentences in the dataframe
all_sentences = []

for text in df['body_content']:
    sentences = split_sentences(text)
    all_sentences.extend(sentences)

# Create a DataFrame for sentences
sentences_df = pd.DataFrame(all_sentences, columns=['sentence'])

# Remove duplicate sentences
sentences_df = sentences_df.drop_duplicates(subset='sentence')

# Mark important sentences
sentences_df['important'] = sentences_df['sentence'].apply(is_important)

# Filter only important sentences
important_sentences_df = sentences_df[sentences_df['important'] == True].drop(columns=['important'])

sentences_df['dutch'] = sentences_df['sentence'].apply(is_dutch)

dutch_sentences_df = sentences_df[sentences_df['dutch'] == True].drop(columns=['dutch'])

In [ ]:
keywords=['browser','menu','contact','2020 antwerpen','sportcentrum','bel','surf','ook interessant','locatie',':','telefoneer','schrijf','website','aanbod','gezin','euro','mail','tel+','@','€','stadsplan','leaflet','gemeentearchief','cookie', 'NL','internetbrowser','E-mail','©','™','|']

pattern = '|'.join(keywords)

important_sentences_df = important_sentences_df[~important_sentences_df['sentence'].str.contains(pattern, case=False)]

In [ ]:
average_length = important_sentences_df['sentence'].str.len().mean()

important_sentences_df = important_sentences_df[important_sentences_df['sentence'].str.len() >= average_length-18]

In [ ]:
important_sentences_df.reset_index(drop=True, inplace=True)

In [ ]:
# Export important sentences to a CSV file
important_sentences_df.to_csv('important_sentences.csv', index=True)
print("Important sentences exported successfully.")

In [ ]:
# Display a few important sentences
important_sentences_df.head()